In [10]:
import pandas as pd
from feature_calculation import build_biomarker_csv

In [11]:
data = pd.read_csv("/Users/thomas.cong/Downloads/Voice_Data/W0000-NEU-CES-2157_Winterlight.csv")

subject_dict = data[["SUBJID", "X_DUID", "VISIT"]].drop_duplicates().set_index("X_DUID").to_dict()
for x in subject_dict["SUBJID"]:
    if subject_dict["VISIT"][x] == "V1":
        subject_dict["SUBJID"][x] += "OFF"
    else:
        subject_dict["SUBJID"][x] += "ON"
subject_dict = {x: subject_dict["SUBJID"][x] for x in subject_dict["SUBJID"]}
on_off_dict = {x: "OFF" if "OFF" in subject_dict[x] else "ON" for x in subject_dict}
subject_only = {x: subject_dict[x].split("O")[0] for x in subject_dict}

In [62]:
biomarkers_2157_with_clinical = pd.read_csv("./2157-Generated-Data/2157-Biomarkers.csv", index_col = "winterlight_id")
biomarkers_2157_with_clinical["subjid"] = biomarkers_2157_with_clinical.index.map(subject_dict)

In [63]:
UPDRS_Scores = pd.read_csv("/Users/thomas.cong/Downloads/Clinical_Data/2157 CES Clinical data from RUSH university/clinical-rush.csv")
UPDRS_Scores = pd.concat([UPDRS_Scores, pd.read_csv("/Users/thomas.cong/Downloads/Clinical_Data/2157 CES Clinical data from RUSH university/clinical-cumc.csv")])
UPDRS_Scores = UPDRS_Scores.pivot(index=["subjid", "state"], columns = ["mds_updrs_label"], values="mds_updrs_value")
UPDRS_Scores.reset_index(inplace=True)
UPDRS_Scores["subjid"] = UPDRS_Scores["subjid"] + UPDRS_Scores["state"]
UPDRS_Scores.drop(columns=["state"], inplace=True)
UPDRS_Scores.set_index("subjid", inplace=True)
UPDRSIII_Mapping = {x: sum(UPDRS_Scores.loc[x]) for x in UPDRS_Scores.index}
Speech_Mapping = {x: int(UPDRS_Scores.loc[x]["Speech"]) for x in UPDRS_Scores.index}
biomarkers_2157_with_clinical["UPDRSIII"] = biomarkers_2157_with_clinical["subjid"].map(UPDRSIII_Mapping)
biomarkers_2157_with_clinical["Speech"] = biomarkers_2157_with_clinical["subjid"].map(Speech_Mapping)
biomarkers_2157_with_clinical["Status"] = "O" + biomarkers_2157_with_clinical["subjid"].str.split("O").str[1]
biomarkers_2157_with_clinical["Status"] = biomarkers_2157_with_clinical["Status"].fillna("HC")

In [64]:
for col in biomarkers_2157_with_clinical.columns:
    if col != "UPDRSII" and col != "Speech":
        try:
            std = biomarkers_2157_with_clinical[col].std()
            mean = biomarkers_2157_with_clinical[col].mean()
            biomarkers_2157_with_clinical[col] = biomarkers_2157_with_clinical[col].apply(lambda x: max(min((x - mean) / std, 3), -3))
        except TypeError:
            pass

In [66]:
biomarkers_2157_with_clinical.to_csv("./2157-Generated-Data/2157-Clinical-Biomarkers-Normalized.csv")